Prueba de progreso 2

Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

Carga de datos

In [ ]:

# Ruta de la carpeta que contiene los ficheros
ruta_carpeta = 'dataset/data'

# Lista para almacenar los DataFrames
dataframes = []

# Recorrer todos los subdirectorios y archivos en la carpeta
for subdir, _, files in os.walk(ruta_carpeta):
    for file in files:
        if file.endswith('.csv'):  # Asegurarse de procesar solo archivos CSV
            # Construir la ruta completa del archivo
            ruta_archivo = os.path.join(subdir, file)
            
            # Extraer información del sujeto y la grabación del nombre del archivo y la ruta
            partes = subdir.split(os.sep)
            sujeto = partes[-1]  # Nombre del sujeto (última carpeta)
            grabacion = file.split('.')[0]  # Nombre de la grabación (sin extensión)
            
            # Cargar el archivo CSV en un DataFrame
            df = pd.read_csv(ruta_archivo)
            
            # Agregar columnas para identificar el sujeto y la grabación
            df['Sujeto'] = sujeto
            df['Grabacion'] = grabacion
            
            # Agregar el DataFrame a la lista
            dataframes.append(df)

# Combinar todos los DataFrames en uno solo
datos_combinados = pd.concat(dataframes, ignore_index=True)

# Mostrar las primeras filas del DataFrame combinado
print(datos_combinados.head())

Eliminar correlación entre articulaciones

In [ ]:
# Copia del DataFrame original
resta = datos_combinados.copy()

# Itera de tres en tres columnas, comenzando desde la cuarta columna
for i in range(3, resta.shape[1], 3):  # Empieza en la columna 3 y avanza de 3 en 3
    resta.iloc[:, i:i+3] -= datos_combinados.iloc[:, 0:3].values  # Resta las columnas X, Y, Z de la pelvis

#Ahora vamos a afinar un poco mas
resta2 = resta.copy()
for j in range(67, 67+3, 1): #Tiene en cuenta las partes del cuerpo que dependen de otras, no solo de la pelvis
    resta2.iloc[:,j] -= resta2.iloc[:, j-3].values
    resta2.iloc[:,j-12] -= resta2.iloc[:, j-3-12].values
    resta2.iloc[:,j-24] -= resta2.iloc[:, j-3-24].values
    resta2.iloc[:,j-36] -= resta2.iloc[:, j-3-36].values

for j in range(19, 19+3, 1):
    resta2.iloc[:,j] -= resta2.iloc[:, j-3].values

Ventaneo

In [ ]:
freq =60
segundos = 4
ventana = freq*segundos
a=[]
for i in range(0,len(resta)-ventana+1, ventana):
  a.append(resta.iloc[i:i+ventana])

Normalización

In [ ]:
# Normalización por persona
def normalizar_por_persona(df):
    """
    Normaliza los puntos de cada persona para que las dimensiones individuales no afecten.
    """
    # Agrupar por sujeto
    grupos = df.groupby('Sujeto')
    
    # Aplicar normalización a cada grupo
    df_normalizado = grupos.transform(lambda x: (x - x.mean()) / x.std())
    
    # Mantener las columnas de identificación
    df_normalizado['Sujeto'] = df['Sujeto']
    df_normalizado['Grabacion'] = df['Grabacion']
    
    return df_normalizado

# Normalización por ventana
def normalizar_por_ventana(df):
    """
    Normaliza las rutas en cada ventana para que todas comiencen desde (x=0, y=0, z=0).
    """
    # Restar la posición inicial de la pelvis a todas las coordenadas
    for i in range(0, df.shape[0], ventana):
        ventana_df = df.iloc[i:i+ventana]
        pelvis_inicial = ventana_df.iloc[0, :3].values  # Coordenadas iniciales de la pelvis
        df.iloc[i:i+ventana, :] -= pelvis_inicial
    
    return df

# Aplicar las normalizaciones
datos_normalizados_persona = normalizar_por_persona(datos_combinados)
datos_normalizados_final = normalizar_por_ventana(datos_normalizados_persona)

# Mostrar las primeras filas del DataFrame normalizado
print(datos_normalizados_final.head())

Extracción de características

In [ ]:
# Función para calcular la energía de la señal
def calcular_energia(serie):
    return np.sum(serie**2)

# Función para calcular los parámetros de Hjorth
def parametros_hjorth(serie):
    derivada_1 = np.diff(serie)
    derivada_2 = np.diff(derivada_1)
    varianza = np.var(serie)
    movilidad = np.sqrt(np.var(derivada_1) / varianza)
    complejidad = np.sqrt(np.var(derivada_2) / np.var(derivada_1)) / movilidad
    return movilidad, complejidad

# Crear un DataFrame para almacenar las características
caracteristicas = []

# Iterar sobre las ventanas de datos
for ventana in a:  # 'a' contiene las ventanas creadas previamente
    # Diccionario para almacenar las características de la ventana actual
    caracteristicas_ventana = {}
    
    # Iterar sobre las columnas de la ventana (asumiendo que están organizadas en bloques de 3: X, Y, Z)
    for i in range(0, ventana.shape[1], 3):
        eje_x = ventana.iloc[:, i]
        eje_y = ventana.iloc[:, i + 1]
        eje_z = ventana.iloc[:, i + 2]
        
        # Calcular características para cada eje
        for eje, nombre in zip([eje_x, eje_y, eje_z], ['X', 'Y', 'Z']):
            caracteristicas_ventana[f'Media_{i // 3}_{nombre}'] = eje.mean()
            caracteristicas_ventana[f'Desviacion_{i // 3}_{nombre}'] = eje.std()
            caracteristicas_ventana[f'Minimo_{i // 3}_{nombre}'] = eje.min()
            caracteristicas_ventana[f'Maximo_{i // 3}_{nombre}'] = eje.max()
            caracteristicas_ventana[f'Energia_{i // 3}_{nombre}'] = calcular_energia(eje)
            movilidad, complejidad = parametros_hjorth(eje)
            caracteristicas_ventana[f'Movilidad_{i // 3}_{nombre}'] = movilidad
            caracteristicas_ventana[f'Complejidad_{i // 3}_{nombre}'] = complejidad
    
    # Agregar las características de la ventana al listado general
    caracteristicas.append(caracteristicas_ventana)

# Convertir las características en un DataFrame final
caracteristicas_df = pd.DataFrame(caracteristicas)

# Mostrar las primeras filas del DataFrame de características
print(caracteristicas_df.head())

Elección de algoritmo

Simplicidad y eficiencia: K-Means es un algoritmo sencillo y eficiente para conjuntos de datos de tamaño moderado, como el que estás manejando.

Datos numéricos: Tus datos (características extraídas como media, desviación típica, energía, etc.) son numéricos, lo que se adapta bien a K-Means, que utiliza distancias euclidianas para agrupar.

Interpretabilidad: Los resultados de K-Means son fáciles de interpretar, ya que cada punto pertenece a un único clúster.

Escalabilidad: K-Means puede manejar un número razonable de ventanas y articulaciones, lo que parece ser el caso en tu proyecto.

Consideraciones:

Número de clústeres: Es necesario determinar el número óptimo de clústeres (k). Esto se puede hacer utilizando el método del codo (elbow method) o el índice de silueta.

Normalización previa: Dado que K-Means es sensible a la escala de los datos, es importante que los datos estén normalizados, lo cual ya has implementado.

In [ ]:


# Determinar el número óptimo de clústeres usando el método del codo
def determinar_numero_clusters(features, max_clusters=10):
    inertias = []
    for k in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(features)
        inertias.append(kmeans.inertia_)
    
    # Graficar el método del codo
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, max_clusters + 1), inertias, marker='o')
    plt.title('Método del Codo')
    plt.xlabel('Número de Clústeres')
    plt.ylabel('Inercia')
    plt.show()

# Aplicar K-Means con el número óptimo de clústeres
def aplicar_kmeans(features, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(features)
    
    # Calcular el puntaje de silueta para evaluar la calidad del clustering
    silhouette_avg = silhouette_score(features, labels)
    print(f'Puntaje de Silueta: {silhouette_avg}')
    
    return labels, kmeans

# Seleccionar las características para el clustering
features = caracteristicas_df.dropna().values  # Asegurarse de que no haya valores nulos

# Determinar el número óptimo de clústeres
determinar_numero_clusters(features, max_clusters=10)

# Aplicar K-Means con un número de clústeres elegido (por ejemplo, 4)
n_clusters = 4
labels, kmeans_model = aplicar_kmeans(features, n_clusters)

# Agregar las etiquetas de clúster al DataFrame de características
caracteristicas_df['Cluster'] = labels

# Mostrar las primeras filas con las etiquetas de clúster
print(caracteristicas_df.head())

Interpretación de resultados

In [ ]:

# Resúmenes estadísticos por clúster
resumen_clusters = caracteristicas_df.groupby('Cluster').mean()
print('Resumen por clúster:')
print(resumen_clusters)

# Visualización de los clústeres en 2D usando PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
features_pca = pca.fit_transform(features)

plt.figure(figsize=(10, 6))
for cluster in range(n_clusters):
    cluster_points = features_pca[caracteristicas_df['Cluster'] == cluster]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Clúster {cluster}')

plt.title('Visualización de Clústeres (PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.show()

# Visualización del movimiento de un clúster específico
def visualizar_movimiento_cluster(cluster_id, data, ventanas):
    cluster_data = data[data['Cluster'] == cluster_id]
    promedio_movimiento = cluster_data.mean(axis=0)  # Promedio de las ventanas del clúster
    
    # Graficar el movimiento promedio
    plt.figure(figsize=(8, 6))
    plt.plot(promedio_movimiento, label=f'Movimiento Promedio Clúster {cluster_id}')
    plt.title(f'Movimiento Promedio del Clúster {cluster_id}')
    plt.xlabel('Tiempo (frames)')
    plt.ylabel('Posición Normalizada')
    plt.legend()
    plt.grid()
    plt.show()

# Ejemplo: Visualizar el movimiento del clúster 0
visualizar_movimiento_cluster(0, caracteristicas_df, a)